In [2]:
import pandas as pd
from utils import get_data_with_filtered_columns

# Preparing data

In [9]:
X, y, final_columns = get_data_with_filtered_columns(top_columns=10, correlation_threshold=0.8)
X

/home/uxeruses/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


,48,64,105,241,282,323,338,424,442,453
0,440,648,181,434,492,476,628,496,568,471
1,499,488,431,551,491,463,528,457,463,311
2,460,485,593,474,485,486,431,474,503,606
3,529,415,698,569,470,458,377,501,447,545
4,429,387,451,538,497,485,509,504,536,426
...,...,...,...,...,...,...,...,...,...,...
1995,505,482,541,535,526,466,419,502,509,570
1996,448,311,696,590,483,478,348,475,421,423
1997,544,540,752,463,474,471,406,480,428,643
1998,490,543,524,443,478,479,473,469,563,691


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

scaler = MinMaxScaler()
encoder = OneHotEncoder(sparse=False)

X = scaler.fit_transform(X)
y = encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42, stratify=y)


# Autosklearn classifier

In [20]:
from autosklearn.metrics import roc_auc, balanced_accuracy
from autosklearn.classification import AutoSklearnClassifier
from sklearn.metrics import accuracy_score

settings = {
  "time_left_for_this_task": 60 * 30,  # 30 minutes in seconds
  "seed": 42,
  "metric": roc_auc,
  "n_jobs": 8,
  "initial_configurations_via_metalearning": 0,
  "resampling_strategy": "cv", 
  "resampling_strategy_arguments": {"folds": 10}
}

automl_classifier = AutoSklearnClassifier(**settings)

# Fit autosklearn classifier
automl_classifier.fit(X_train, y_train)

# Predict with autosklearn classifier
y_pred = automl_classifier.predict(X_test)

# Model accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

[WARNING] [2024-01-14 12:09:22,613:Client-AutoML(42):53d16ce6-b2cd-11ee-b867-4f6762ecf133] Capping the per_run_time_limit to 899.0 to have time for a least 2 models in each process.
Test Accuracy: 0.878


# Exporting final data

In [40]:
x_final = pd.read_csv("data/artificial_test.data", sep=' ', header=None)
x_final = x_final.drop(x_final.columns[-1], axis=1)

x_final = x_final[final_columns]

x_final = scaler.transform(x_final)

In [41]:
y_final_proba = automl_classifier.predict_proba(x_final)
y_final_aml = y_final_proba[:, 1]

In [42]:
pd.DataFrame(y_final_proba).head()

,0,1
0,0.834382,0.165618
1,0.891218,0.108782
2,0.442964,0.557036
3,0.060313,0.939687
4,0.632229,0.367771


In [43]:
import numpy as np

np.savetxt('data/313180_313282_artifical_automl_prediction.txt', y_final_aml, delimiter=',', fmt='%s')